In [1]:
# Google cloud
from google.cloud import datastore
from google.cloud import storage
import traceback
from django.http import HttpResponse
import json
import pandas as pd
import numpy as np
import datetime
import math
import matplotlib.pyplot as plt

In [2]:
storage_client = storage.Client('eps-server')
datastore_client = datastore.Client('eps-server')

/opt/homebrew/Caskroom/miniforge/base/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/homebrew/Caskroom/miniforge/base/lib/python3.8/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For mor

In [257]:
devices = {
    'b8:27:eb:b7:23:e3': '5081', 'b8:27:eb:74:f3:03': '1934',
    'b8:27:eb:ba:15:05': '1884', 'b8:27:eb:a7:8b:ff': '1064', 'b8:27:eb:15:88:2c': '1695', 'b8:27:eb:fd:af:6d': '1699',
    'b8:27:eb:1c:2f:04': '5029', 'b8:27:eb:12:c4:6c': '5031', 'b8:27:eb:14:0d:9a': '5033', 'b8:27:eb:50:1a:c2': '5046',
    'b8:27:eb:2f:60:9e': '5069', 'b8:27:eb:ab:dd:49': '5148', 'b8:27:eb:69:9f:b7': '5172', 'b8:27:eb:20:3d:2b': '5199',
    'b8:27:eb:21:77:56': '5007', 'b8:27:eb:9e:ab:0f': '1093',
    'b8:27:eb:03:9c:af': '5027',
    'b8:27:eb:5f:b1:34': '5141',
    'b8:27:eb:92:6f:f2': '5150',
    'b8:27:eb:a6:b3:aa': '5175',
    'b8:27:eb:0e:22:a2': '5176',
    'b8:27:eb:1f:8d:37': '5178',
    'b8:27:eb:a2:75:f1': '5179',
    'b8:27:eb:1d:7a:4f': '5184',
    'b8:27:eb:1d:94:59': '5190'
}

In [3]:
def get_version(devices):
    version = []
    query = datastore_client.query(kind='deviceregister')
    for device in devices:
        #print(device)
        key = datastore_client.key('deviceregister', device)
        query.key_filter(key, "=")
        try:
            version.append([{key:val for key,val in _.items()} for _ in query.fetch()][0]['release-id'])
        except:
            pass    
    return version

In [230]:
devices = ''
update = []
query = datastore_client.query(kind='updateDevice')
dlist = list(devices.keys())
dlist.sort()
for device in dlist:   
    key = datastore_client.key('updateDevice', device)
    task_fleet = datastore_client.get(key)
    task_fleet['update'] = 'yes'
    datastore_client.put(task_fleet)
update

[]

In [216]:
devices = ''
update = []
query = datastore_client.query(kind='updateDevice')
nolist = list(devices.keys())
nolist.sort()
for device in nolist:   
    key = datastore_client.key('updateDevice', device)
    task_fleet = datastore_client.get(key)
    task_fleet['update'] = 'no'
    datastore_client.put(task_fleet)
update

[]

# 

In [258]:
update = []
query = datastore_client.query(kind='deviceregister')

dlist = list(devices.keys())
dlist.sort()
table = []
online = {'pi-up':'ONLINE','pi-down':'offline'}
for device in dlist:   
    register_key = datastore_client.key('deviceregister', device)
    task_fleet = datastore_client.get(register_key)
    last_key = datastore_client.key('last', device)
    last = datastore_client.get(last_key)
    
    key = datastore_client.key('updateDevice', device)
    update = datastore_client.get(key)
    
    table.append({'name':devices[device],'macID':device,'last-updated':task_fleet['last-updated'],
                  'current version':task_fleet['release-id'].split('-V')[-1],
                  'downloaded version':task_fleet['unique-id'],'last online status':online[last['system-status']],'currently flagged':update['update']})
    #print(devices[device]+' : '+device+' - '+task_fleet['last-updated']+' - '+task_fleet['release-id']+' - '+task_fleet['unique-id']+' - ')
update
update_df = pd.DataFrame(table)
update_df.sort_values('current version').sort_values('macID')#.to_excel('update_status.xlsx')

,name,macID,last-updated,current version,downloaded version,last online status,currently flagged
0,5027,b8:27:eb:03:9c:af,12/17/20 07:34:23,2.8B,f643c802d2d9514b90b95decd3a449d7,offline,yes
1,5176,b8:27:eb:0e:22:a2,12/17/20 13:47:38,2.8B,f643c802d2d9514b90b95decd3a449d7,offline,yes
2,5031,b8:27:eb:12:c4:6c,04/06/21 19:48:07,2.8.2,943c34aca1f65bf89648f3f2f503ddcd,offline,no
3,5033,b8:27:eb:14:0d:9a,04/05/21 11:48:57,2.8.2,943c34aca1f65bf89648f3f2f503ddcd,offline,no
4,1695,b8:27:eb:15:88:2c,04/05/21 15:44:23,2.8.2,943c34aca1f65bf89648f3f2f503ddcd,offline,no
5,5029,b8:27:eb:1c:2f:04,04/04/21 19:31:45,2.8.2,943c34aca1f65bf89648f3f2f503ddcd,offline,no
6,5184,b8:27:eb:1d:7a:4f,03/22/21 07:44:36,2.8F,c7b82b2ae52557139c5e490e31e26ee4,offline,yes
7,5190,b8:27:eb:1d:94:59,03/23/21 08:45:50,2.8.1,0e3564a086eb54e6a5a8373673e572fd,offline,yes
8,5178,b8:27:eb:1f:8d:37,09/17/20 16:51:30,2.7B,d23557d1b2e75293a4ff78c872f8764b,offline,yes
9,5199,b8:27:eb:20:3d:2b,04/04/21 19:55:24,2.8.2,943c34aca1f65bf89648f3f2f503ddcd,offline,no


In [128]:
update = []
query = datastore_client.query(kind='updateDevice')
dlist = ['b8:27:eb:21:77:56']#list(devices.keys())
dlist.sort()
for device in dlist:   
    key = datastore_client.key('updateDevice', device)
    task_fleet = datastore_client.get(key)
    print(device+' - '+task_fleet['update'])
update

b8:27:eb:21:77:56 - yes


[]

In [108]:
update_df.to_excel('update_status.xlsx')

In [197]:
yes_list = update_df[update_df['current version'] != '2.8.2']['macID'].tolist()

In [214]:
update_df.sort_values('current version').sort_values('currently flagged').to_csv('update.csv')

In [260]:
import json
json.dumps("{'b8:27:eb:1d:7a:4f': '5184','b8:27:eb:1d:94:59': '5190'}")

'"{\'b8:27:eb:1d:7a:4f\': \'5184\',\'b8:27:eb:1d:94:59\': \'5190\'}"'

In [ ]:
update = []
query = datastore_client.query(kind='deviceregister')

dlist = list(devices.keys())
dlist.sort()
table = []
online = {'pi-up':'ONLINE','pi-down':'offline'}
for device in dlist:   
    register_key = datastore_client.key('deviceregister', device)
    task_fleet = datastore_client.get(register_key)
    last_key = datastore_client.key('last', device)
    last = datastore_client.get(last_key)
    
    key = datastore_client.key('updateDevice', device)
    update = datastore_client.get(key)
    
    table.append({'name':devices[device],'macID':device,'last-updated':task_fleet['last-updated'],
                  'current version':task_fleet['release-id'].split('-V')[-1],
                  'downloaded version':task_fleet['unique-id'],'last online status':online[last['system-status']],'currently flagged':update['update']})
    #print(devices[device]+' : '+device+' - '+task_fleet['last-updated']+' - '+task_fleet['release-id']+' - '+task_fleet['unique-id']+' - ')
update
update_df = pd.DataFrame(table)
update_df.sort_values('current version').sort_values('macID')#.to_excel('update_status.xlsx')

In [255]:
query = datastore_client.query(kind='deviceregister')
query.add_filter('release-id', '=', 'EPSEMS2-V2.8.2')
flagged = [device.key.name for device in list(query.fetch())]
query = datastore_client.query(kind='deviceregister')
query.add_filter('release-id', '=', 'EPSEMS2-V2.9.2')
flagged = flagged + [device.key.name for device in list(query.fetch())]

16


42

In [144]:
list(devices.keys())

['b8:27:eb:b7:23:e3']